<a href="https://colab.research.google.com/github/halimcan/Home-Credit-Default-Project/blob/branch3/bureau_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installments of required tables
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
#  BigQuery API activation
from google.colab import auth
auth.authenticate_user()


In [3]:
# big query add-on installation

!pip install --quiet google-cloud-bigquery
from google.cloud import bigquery

In [4]:
# BigQuery client initiation
client = bigquery.Client(project="homecredit-478707")

# From BigQuery
query = """
SELECT *
FROM `homecredit-478707.Homecredit_Tables.bureau`
"""
bureau = client.query(query).to_dataframe()

In [5]:
bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              Int64  
 1   SK_ID_BUREAU            Int64  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             Int64  
 5   CREDIT_DAY_OVERDUE      Int64  
 6   DAYS_CREDIT_ENDDATE     float64
 7   DAYS_ENDDATE_FACT       float64
 8   AMT_CREDIT_MAX_OVERDUE  float64
 9   CNT_CREDIT_PROLONG      Int64  
 10  AMT_CREDIT_SUM          float64
 11  AMT_CREDIT_SUM_DEBT     float64
 12  AMT_CREDIT_SUM_LIMIT    float64
 13  AMT_CREDIT_SUM_OVERDUE  float64
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      Int64  
 16  AMT_ANNUITY             float64
dtypes: Int64(6), float64(8), object(3)
memory usage: 232.4+ MB


In [6]:
bureau.describe()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
count,1716428.0,1716428.0,1716428.0,1716428.0,1.610875e+06,1.082775e+06,5.919400e+05,1716428.0,1.716415e+06,1.458759e+06,1.124648e+06,1.716428e+06,1716428.0,4.896370e+05
mean,278214.933645,5924434.489032,-1142.107685,0.818167,5.105174e+02,-1.017437e+03,3.825418e+03,0.00641,3.549946e+05,1.370851e+05,6.229515e+03,3.791276e+01,-593.74832,1.571276e+04
std,102938.558112,532265.728552,795.164928,36.544428,4.994220e+03,7.140106e+02,2.060316e+05,0.096224,1.149811e+06,6.774011e+05,4.503203e+04,5.937650e+03,720.747312,3.258269e+05
min,100001.0,5000000.0,-2922.0,0.0,-4.206000e+04,-4.202300e+04,0.000000e+00,0.0,0.000000e+00,-4.705600e+06,-5.864061e+05,0.000000e+00,-41947.0,0.000000e+00
25%,188866.75,5463953.75,-1666.0,0.0,-1.138000e+03,-1.489000e+03,0.000000e+00,0.0,5.130000e+04,0.000000e+00,0.000000e+00,0.000000e+00,-908.0,0.000000e+00
50%,278055.0,5926303.5,-987.0,0.0,-3.300000e+02,-8.970000e+02,0.000000e+00,0.0,1.255185e+05,0.000000e+00,0.000000e+00,0.000000e+00,-395.0,0.000000e+00
75%,367426.0,6385681.25,-474.0,0.0,4.740000e+02,-4.250000e+02,0.000000e+00,0.0,3.150000e+05,4.015350e+04,0.000000e+00,0.000000e+00,-33.0,1.350000e+04
max,456255.0,6843457.0,0.0,2792.0,3.119900e+04,0.000000e+00,1.159872e+08,9.0,5.850000e+08,1.701000e+08,4.705600e+06,3.756681e+06,372.0,1.184534e+08


# Primary Key Check

In [37]:
bureau['SK_ID_BUREAU'].is_unique    # Must be true

True

In [36]:
bureau['SK_ID_CURR'].is_unique    # Must be false (normal)

False

## Credit Behavior Insights (Domain EDA)

These are very important for credit scoring:

Percentage of loans in debt → powerful predictor

Percentage of loans at their credit limit → indicates high risk

Presence of any active loans → very important signal


These tells us which categories could be encoded.

In [35]:
bureau['CREDIT_ACTIVE'].value_counts()



,count
CREDIT_ACTIVE,
Closed,1079273
Active,630607
Sold,6527
Bad debt,21


In [34]:
bureau['CREDIT_TYPE'].value_counts()

,count
CREDIT_TYPE,
Consumer credit,1251615
Credit card,402195
Car loan,27690
Mortgage,18391
Microloan,12413
Loan for business development,1975
Another type of loan,1017
Unknown type of loan,555
Loan for working capital replenishment,469


In [8]:
# Check that SK_ID_CURR exists and how many unique customers

if 'SK_ID_CURR' in bureau.columns:
    total_rows = len(bureau)
    unique_customers = bureau['SK_ID_CURR'].nunique()
    print(f"Total bureau rows: {total_rows}, Unique customers: {unique_customers}")

Total bureau rows: 1716428, Unique customers: 305811


## # Most critical EDA output is :

In [38]:

#How many external loans does each customer have?

bureau.groupby('SK_ID_CURR').size()

# then this goes into aggregated bureau as bureau_total_loans


,0
SK_ID_CURR,
100001,7
100002,8
100003,4
100004,2
100005,3
...,...
456249,13
456250,3
456253,4


In [9]:
bureau.isnull().mean().sort_values(ascending=False)
# AMT_ANNUITY,AMT_CREDIT_MAX_OVERDUE, DAYS_ENDDATE_FACT, AMT_CREDIT_SUM_LIMIT variables have high missing value ratios.

# These are loan level missings which is pretty normal.

# mean, min, max → could work!!

# count of non-nulls + missing rate → could be strong features.



,0
AMT_ANNUITY,0.714735
AMT_CREDIT_MAX_OVERDUE,0.655133
DAYS_ENDDATE_FACT,0.369170
AMT_CREDIT_SUM_LIMIT,0.344774
AMT_CREDIT_SUM_DEBT,0.150119
DAYS_CREDIT_ENDDATE,0.061496
AMT_CREDIT_SUM,0.000008
SK_ID_CURR,0.000000
SK_ID_BUREAU,0.000000
CREDIT_DAY_OVERDUE,0.000000


In [10]:
# Create missing-value indicator flags in order for LightGBM algorithm to work better.

# For columns that are informative when missing, create *_MISSING flags
missing_cols = [
    "AMT_ANNUITY",               # 71% missing -> create flag
    "AMT_CREDIT_MAX_OVERDUE",    # 65% missing -> create flag
    "DAYS_ENDDATE_FACT",         # ~37% missing -> create flag
    "AMT_CREDIT_SUM_LIMIT",      # ~34% missing -> create flag (credit card limit)
    "AMT_CREDIT_SUM_DEBT",       # ~15% missing -> create flag
    "DAYS_CREDIT_ENDDATE"        # ~6% missing -> optional flag
]

for col in missing_cols:
    if col in bureau.columns:
        # create binary missing flag (1 = missing)
        bureau[col + "_MISSING"] = bureau[col].isna().astype(int)
    else:
        print(f"Warning: {col} not in bureau columns, skipping missing flag creation.")

In [11]:
# Small-fill for nearly-complete numeric columns

# AMT_CREDIT_SUM is almost complete (0.0008 missing). Fill with median to avoid NaNs in aggregations.
if 'AMT_CREDIT_SUM' in bureau.columns:
    n_missing_amt_sum = bureau['AMT_CREDIT_SUM'].isna().sum()
    if n_missing_amt_sum > 0:
        median_val = bureau['AMT_CREDIT_SUM'].median()
        # fill with median (safe because missing is negligible)
        bureau['AMT_CREDIT_SUM'] = bureau['AMT_CREDIT_SUM'].fillna(median_val)
        print(f"Filled {n_missing_amt_sum} missing AMT_CREDIT_SUM with median = {median_val:.2f}")

Filled 13 missing AMT_CREDIT_SUM with median = 125518.50


In [12]:
# Convert some columns to numeric if they are stored as objects

# This prevents skipping numeric operations
to_numeric_cols = [
    "DAYS_CREDIT", "DAYS_CREDIT_ENDDATE", "DAYS_ENDDATE_FACT",
    "AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_LIMIT",
    "AMT_CREDIT_MAX_OVERDUE", "AMT_ANNUITY", "CNT_CREDIT_PROLONG",
    "DAYS_CREDIT_UPDATE"
]
for col in to_numeric_cols:
    if col in bureau.columns:
        bureau[col] = pd.to_numeric(bureau[col], errors='coerce')

## Credit Behavior Profiling

CREDIT_ACTIVE → types.

CREDIT_TYPE → types

From loan-level data, i derived customer-level summary features:

Total debt → bureau_total_debt

Maximum overdue amount → bureau_max_overdue_amount

Number of active loans → bureau_total_active_loans

In [13]:
# Feature engineering: aggregate per SK_ID_CURR

# I'll produce a set of robust, interpretable features commonly used in credit risk.

agg_funcs = {
    # counts
    'SK_ID_BUREAU': ['count'],  # number of external credits per customer
    # credit amounts
    'AMT_CREDIT_SUM': ['sum', 'mean', 'max'],
    'AMT_CREDIT_SUM_DEBT': ['sum', 'mean'],
    'AMT_CREDIT_SUM_LIMIT': ['mean'],  # avg credit limit (mostly for cards)
    'AMT_CREDIT_MAX_OVERDUE': ['max'], # worst overdue amount
    'AMT_ANNUITY': ['mean'],
    # days / durations
    'DAYS_CREDIT': ['min','max','mean'],
    'DAYS_CREDIT_ENDDATE': ['min','max','mean'],
    'DAYS_ENDDATE_FACT': ['min','max','mean'],
    'DAYS_CREDIT_UPDATE': ['min','max','mean'],
    # counts of categorical-type signals
    'CNT_CREDIT_PROLONG': ['sum','max']
}

In [14]:
# Only keep keys present in dataframe to avoid KeyError

agg_funcs_filtered = {k:v for k,v in agg_funcs.items() if k in bureau.columns}

In [15]:
# Perform groupby aggregation
bureau_agg = bureau.groupby('SK_ID_CURR').agg(agg_funcs_filtered)

In [16]:
bureau_agg.head(3)

SK_ID_BUREAU AMT_CREDIT_SUM                           \
                  count            sum           mean       max   
SK_ID_CURR                                                        
100001                7    1453365.000  207623.571429  378000.0   
100002                8     865055.565  108131.945625  450000.0   
100003                4    1017400.500  254350.125000  810000.0   

           AMT_CREDIT_SUM_DEBT               AMT_CREDIT_SUM_LIMIT  \
                           sum          mean                 mean   
SK_ID_CURR                                                          
100001                596686.5  85240.928571              0.00000   
100002                245781.0  49156.200000           7997.14125   
100003                     0.0      0.000000         202500.00000   

           AMT_CREDIT_MAX_OVERDUE  AMT_ANNUITY DAYS_CREDIT  ...  \
                              max         mean         min  ...   
SK_ID_CURR                                                  ...   
100001                        NaN  3545.357143       -1572  ...   
100002                   5043.645     0.000000       -1437  ...   
100003                      0.000          NaN       -2586  ...   

           DAYS_CREDIT_ENDDATE             DAYS_ENDDATE_FACT         \
                           max        mean               min    max   
SK_ID_CURR                                                            
100001                  1778.0   82.428571           -1328.0 -544.0   
100002                   780.0 -349.000000           -1185.0  -36.0   
100003                  1216.0 -544.500000           -2131.0 -540.0   

                        DAYS_CREDIT_UPDATE                 CNT_CREDIT_PROLONG  \
                   mean                min  max       mean                sum   
SK_ID_CURR                                                                      
100001      -825.500000               -155   -6 -93.142857                  0   
100002      -697.500000              -1185   -7   -499.875                  0   
100003     -1097.333333              -2131  -43     -816.0                  0   

                
           max  
SK_ID_CURR      
100001       0  
100002       0  
100003       0  

[3 rows x 23 columns]

In [17]:
# Flatten multiindex columns
bureau_agg.columns = ['_'.join(col).strip() for col in bureau_agg.columns.values]

In [18]:
# Rename some columns for clarity
rename_map = {
    'SK_ID_BUREAU_count': 'bureau_total_loans',
    'AMT_CREDIT_SUM_sum': 'bureau_total_credit_amount',
    'AMT_CREDIT_SUM_mean': 'bureau_mean_credit_amount',
    'AMT_CREDIT_SUM_max': 'bureau_max_credit_amount',
    'AMT_CREDIT_SUM_DEBT_sum': 'bureau_total_debt',
    'AMT_CREDIT_SUM_DEBT_mean': 'bureau_mean_debt',
    'AMT_CREDIT_SUM_LIMIT_mean': 'bureau_mean_credit_limit',
    'AMT_CREDIT_MAX_OVERDUE_max': 'bureau_max_overdue_amount',
    'AMT_ANNUITY_mean': 'bureau_mean_annuity',
    'DAYS_CREDIT_min': 'bureau_earliest_credit_days',
    'DAYS_CREDIT_max': 'bureau_latest_credit_days',
    'DAYS_CREDIT_mean': 'bureau_mean_days_credit',
    'DAYS_CREDIT_ENDDATE_min': 'bureau_min_credit_enddate',
    'DAYS_CREDIT_ENDDATE_max': 'bureau_max_credit_enddate',
    'DAYS_CREDIT_ENDDATE_mean': 'bureau_mean_credit_enddate',
    'DAYS_ENDDATE_FACT_min': 'bureau_min_enddate_fact',
    'DAYS_ENDDATE_FACT_max': 'bureau_max_enddate_fact',
    'DAYS_ENDDATE_FACT_mean': 'bureau_mean_enddate_fact',
    'DAYS_CREDIT_UPDATE_min': 'bureau_min_days_credit_update',
    'DAYS_CREDIT_UPDATE_max': 'bureau_max_days_credit_update',
    'DAYS_CREDIT_UPDATE_mean': 'bureau_mean_days_credit_update',
    'CNT_CREDIT_PROLONG_sum': 'bureau_total_prolongs',
    'CNT_CREDIT_PROLONG_max': 'bureau_max_prolongs'
}

# apply rename for keys that exist
rename_map = {k:v for k,v in rename_map.items() if k in bureau_agg.columns}
bureau_agg = bureau_agg.rename(columns=rename_map)

In [19]:
# Additional aggregated indicators

# number of active credits (CREDIT_ACTIVE != 'Closed'), number of closed credits

if 'CREDIT_ACTIVE' in bureau.columns:
    act = bureau.groupby('SK_ID_CURR')['CREDIT_ACTIVE'].apply(
        lambda x: (x != 'Closed').sum()
    ).rename('bureau_active_credits')
    closed = bureau.groupby('SK_ID_CURR')['CREDIT_ACTIVE'].apply(
        lambda x: (x == 'Closed').sum()
    ).rename('bureau_closed_credits')
    bureau_agg = bureau_agg.join(act).join(closed)

In [20]:
bureau_agg.head(2)

,bureau_total_loans,bureau_total_credit_amount,bureau_mean_credit_amount,bureau_max_credit_amount,bureau_total_debt,bureau_mean_debt,bureau_mean_credit_limit,bureau_max_overdue_amount,bureau_mean_annuity,bureau_earliest_credit_days,...,bureau_min_enddate_fact,bureau_max_enddate_fact,bureau_mean_enddate_fact,bureau_min_days_credit_update,bureau_max_days_credit_update,bureau_mean_days_credit_update,bureau_total_prolongs,bureau_max_prolongs,bureau_active_credits,bureau_closed_credits
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,7,1453365.000,207623.571429,378000.0,596686.5,85240.928571,0.00000,NaN,3545.357143,-1572,...,-1328.0,-544.0,-825.5,-155,-6,-93.142857,0,0,3,4
100002,8,865055.565,108131.945625,450000.0,245781.0,49156.200000,7997.14125,5043.645,0.000000,-1437,...,-1185.0,-36.0,-697.5,-1185,-7,-499.875,0,0,2,6


In [21]:
# worst credit status (e.g., any overdue count) by mapping status columns if present
if 'AMT_CREDIT_MAX_OVERDUE' in bureau.columns:
    # fraction of loans with max overdue > 0
    overdue_flag = (bureau['AMT_CREDIT_MAX_OVERDUE'] > 0).astype(int)
    overdue_frac = overdue_flag.groupby(bureau['SK_ID_CURR']).mean().rename('bureau_frac_loans_with_overdue')
    bureau_agg = bureau_agg.join(overdue_frac)

In [22]:
# Aggregate missing flags: percentage of bureau records missing each field
for col in missing_cols:
    flag_col = col + "_MISSING"
    if flag_col in bureau.columns:
        pct_missing = bureau.groupby('SK_ID_CURR')[flag_col].mean().rename(flag_col + "_ratio")
        # e.g., AMT_ANNUITY_MISSING_ratio = fraction of that customer's bureau records missing annuity
        bureau_agg = bureau_agg.join(pct_missing)

In [23]:
# Derived ratios and features
# debt to credit ratio (sum debt / sum credit) - safe with small epsilon
if 'bureau_total_debt' in bureau_agg.columns and 'bureau_total_credit_amount' in bureau_agg.columns:
    eps = 1e-9
    bureau_agg['bureau_debt_to_credit_ratio'] = bureau_agg['bureau_total_debt'] / (bureau_agg['bureau_total_credit_amount'] + eps)


In [24]:
# average loan age (in days) approx using mean DAYS_CREDIT
if 'bureau_mean_days_credit' in bureau_agg.columns:
    bureau_agg['bureau_avg_loan_age_days'] = bureau_agg['bureau_mean_days_credit']

In [25]:
# Final cleanup: fill any inf / extremely tiny NaNs produced
bureau_agg = bureau_agg.replace([np.inf, -np.inf], np.nan)


In [26]:
# For features where missing is negligible, fill with 0
fill_zero_cols = [c for c in bureau_agg.columns if bureau_agg[c].isna().sum() / len(bureau_agg) < 0.05]
bureau_agg[fill_zero_cols] = bureau_agg[fill_zero_cols].fillna(0)

In [27]:
# Output
print("Aggregated bureau features shape:", bureau_agg.shape)
# Optionally save:
bureau_agg.to_csv("bureau_agg.csv", index=True)

# bureau_agg is now customer-level features to merge into master table by SK_ID_CURR

Aggregated bureau features shape: (305811, 34)


While merging I need to use this : # app_train = app_train.merge(bureau_agg, on='SK_ID_CURR', how='left')


Bureau Dataset – Key Points for Feature Engineering

Loan-level dataset → not customer-level

Each client has multiple external credit records

Classical EDA (histograms, outliers, correlations) is not meaningful

Focus on key tasks to support feature engineering:

Primary key validation (SK_ID_BUREAU uniqueness)

Missing value structure analysis (patterns, counts, predictive missingness)

Credit behavior profiling (CREDIT_ACTIVE, CREDIT_TYPE, overdue counts)

Aggregated customer-level features created:

Total debt (bureau_total_debt)

Number of active loans (bureau_total_loans)

Maximum overdue amounts (bureau_max_overdue_amount)

Why We Do Not Apply Full Classical EDA on the Bureau Dataset

The bureau dataset represents loan-level historical credit records from external institutions.
Each customer (SK_ID_CURR) can have multiple bureau records (SK_ID_BUREAU).
Therefore, this table is not customer-level, but transaction-level.

# Why This Approach Works

After aggregating the bureau dataset by SK_ID_CURR, the model receives customer-level summary features such as:

bureau_total_loans

bureau_total_debt

bureau_mean_credit_limit

bureau_max_overdue_amount

These features carry real predictive power and integrate cleanly into the training dataset.